<a href="https://colab.research.google.com/github/Shinde-iru1920/images/blob/main/Real_ESRGAN_Image_and_Video_Upscaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
!pip install tqdm

from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 720):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 39.62 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-

In [3]:
import os
import subprocess

# Define the path to the file
# Use pip show to find the installation location of basicsr
try:
    # Run pip show basicsr and capture the output
    result = subprocess.run(['pip', 'show', 'basicsr'], capture_output=True, text=True, check=True)
    # Find the 'Location:' line in the output
    for line in result.stdout.splitlines():
        if line.startswith('Location:'):
            package_location = line.split(': ')[1].strip()
            break
    else:
        raise Exception("Could not find installation location for basicsr")

    # Construct the file path using the found location
    file_path = os.path.join(package_location, 'basicsr', 'data', 'degradations.py')
    print(f"Constructed file path: {file_path}")

except Exception as e:
    print(f"Error finding basicsr installation location: {e}")
    # Exit or handle the error appropriately
    exit()


# Check if the file exists before attempting to open it
if not os.path.exists(file_path):
    print(f"Error: File not found at the constructed path: {file_path}")
    # Exit or handle the error appropriately
    exit()

# Read the content of the file
with open(file_path, 'r') as file:
    file_data = file.readlines()

# Modify the specific line
modified = False
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        modified = True
        break

if modified:
    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.writelines(file_data)
    print("Modification applied successfully.")
else:
    print("Target line not found in the file. No modification applied.")

Constructed file path: /usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py
Modification applied successfully.


In [4]:
import os
import subprocess
from tqdm import tqdm  # progress bar
import time

# === 🔧 USER SETTINGS ===
input_folder = "/content/drive/MyDrive/Manga/demon slayer manga vol.17 cha- 151.5 - 160.5"  # @param {type:"string"}
output_folder = "/content/drive/MyDrive/Manga/demon slayer manga vol.17 cha- 151.5 - 160.5 upscaled"  # @param {type:"string"}
model = "RealESRGAN_x4plus_anime_6B"  # @param ["RealESRGAN_x4plus", "RealESRGAN_x2plus", "RealESRNet_x4plus", "official ESRGAN_x4", "realesr-general-x4v3", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]
scale = 4  # @param {type:"slider", min:1, max:4, step:1}
face_enhance = False  # @param {type:"boolean"}
code = "inference_realesrgan.py"

# === 📁 SETUP ===
os.makedirs(output_folder, exist_ok=True)
image_extensions = (".png", ".jpg", ".jpeg", ".webp")

# Gather input files (sorted for consistency)
input_files = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(image_extensions)])

# === 🖼️ BATCH INFO ===
print(f"🔍 Found {len(input_files)} image(s) in input folder.")
print(f"🚀 Starting batch upscale to: {output_folder}\n")

# Track failures
failed_files = []

# === 🔄 PROCESS IMAGES ===
for i, filename in enumerate(tqdm(input_files, desc="📈 Upscaling", unit="img"), 1):
    input_path = os.path.join(input_folder, filename)

    command = f"python {code} -i '{input_path}' -n {model} -o '{output_folder}' -s {scale}"
    if face_enhance:
        command += " --face_enhance"

    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed: {filename} — {e}")
        failed_files.append(filename)

    time.sleep(0.1)  # optional system cooldown

# === ✅ SUMMARY ===
print(f"\n✅ Done! {len(input_files) - len(failed_files)}/{len(input_files)} images processed successfully.")

if failed_files:
    print(f"\n❌ {len(failed_files)} image(s) failed to process:")
    for f in failed_files:
        print(" -", f)


🔍 Found 215 image(s) in input folder.
🚀 Starting batch upscale to: /content/drive/MyDrive/Manga/demon slayer manga vol.17 cha- 151.5 - 160.5 upscaled



📈 Upscaling: 100%|██████████| 215/215 [42:09<00:00, 11.77s/img]


✅ Done! 215/215 images processed successfully.


In [6]:
import os
import subprocess
import time
from datetime import datetime

# === User parameters (same as batch cell) ===
input_folder = "/content/drive/MyDrive/Manga/demon slayer manga vol.17 cha- 151.5 - 160.5"
output_folder = "/content/drive/MyDrive/Manga/demon slayer manga vol.17 cha- 151.5 - 160.5 upscaled"
model = "RealESRGAN_x4plus_anime_6B"
scale = 4
face_enhance = False
code = "inference_realesrgan.py"
image_extensions = (".png", ".jpg", ".jpeg", ".webp")

# === Prepare file lists ===
input_files = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(image_extensions)])
output_files = set(os.listdir(output_folder))

# Log file path with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_path = os.path.join(output_folder, f"process_log_{timestamp}.txt")

processed_files = []
missing_outputs = []
failed_files = []

# === Check which files are missing output ===
for filename in input_files:
    name, ext = os.path.splitext(filename)
    expected_output = f"{name}_out{ext}"
    if expected_output in output_files:
        processed_files.append(filename)
    else:
        missing_outputs.append(filename)

# === Retry processing missing files ===
if missing_outputs:
    print(f"\n🔄 Retrying {len(missing_outputs)} missing images...\n")
    for i, filename in enumerate(missing_outputs, 1):
        input_path = os.path.join(input_folder, filename)

        if face_enhance:
            command = f"python {code} -i '{input_path}' -n {model} -o '{output_folder}' -s {scale} --face_enhance"
        else:
            command = f"python {code} -i '{input_path}' -n {model} -o '{output_folder}' -s {scale}"

        print(f"Retrying ({i}/{len(missing_outputs)}): {filename}")
        try:
            subprocess.run(command, shell=True, check=True)
            print(f"✅ Success: {filename}")
            processed_files.append(filename)
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed again: {filename} | Error: {e}")
            failed_files.append(filename)
        time.sleep(0.2)  # cooldown
else:
    print("\n👍 No missing images detected, no retry needed.")

# === Update missing outputs list after retry ===
missing_outputs = [f for f in missing_outputs if f not in processed_files]

# === Write log file ===
with open(log_path, "w") as log:
    log.write(f"Real-ESRGAN Post-Process & Retry Log — {datetime.now()}\n")
    log.write(f"Input folder: {input_folder}\nOutput folder: {output_folder}\n\n")

    log.write("--- Processed Successfully ---\n")
    for f in sorted(processed_files):
        log.write(f"{f}\n")

    log.write("\n--- Missing Output Files ---\n")
    for f in missing_outputs:
        log.write(f"{f}\n")

    if failed_files:
        log.write("\n--- Failed During Retry ---\n")
        for f in failed_files:
            log.write(f"{f}\n")

# === Final summary ===
print(f"\n✅ Verified {len(processed_files)} images processed successfully.")
print(f"⚠️ Missing outputs for {len(missing_outputs)} images.")
if failed_files:
    print(f"❌ Failed to process {len(failed_files)} images even after retry.")
print(f"📁 Log saved at: {log_path}")



🔄 Retrying 4 missing images...

Retrying (1/4): c152_03.jpg
✅ Success: c152_03.jpg
Retrying (2/4): c152_13.jpg
✅ Success: c152_13.jpg
Retrying (3/4): c152_14.jpg
✅ Success: c152_14.jpg
Retrying (4/4): c152_19.jpg
✅ Success: c152_19.jpg

✅ Verified 215 images processed successfully.
⚠️ Missing outputs for 0 images.
📁 Log saved at: /content/drive/MyDrive/Manga/demon slayer manga vol.17 cha- 151.5 - 160.5 upscaled/process_log_2025-05-24_10-04-29.txt
